`Created by Tazein Shah` \
`Last modified on 12/27/2023` \
`This code is the test for using the bool type in julia before I apply it to the whole patient sample (here there is only 20 samples)`

In [26]:
#loading the packages#

#using Pkg
#Pkg.add("DataFrames")
#Pkg.add("Random")
#Pkg.add("RollingFunctions")
#Pkg.add("XLSX")
#Pkg.add("Distributions") 

using DataFrames
using Random
using RollingFunctions
import XLSX
using Distributions

In [27]:
# importing the dataframe #
#this one has the first 20 rows only (it does not have headers)
s7_data_combined = XLSX.readxlsx("/path/julia_test_s7.xlsx")
sh = s7_data_combined["Sheet1"]

20×2 XLSX.Worksheet: ["Sheet1"](A1:B20) 

In [28]:
# setting the variables and constants #

n_obs = 80000  # number of steps
n_constant_nodes = 60000 #number of steps for constant nodes
p = 81  # number of variables
chance_to_flip = 0.75

#creates an "empty" time series
empty_series = falses(p, n_obs)

# Initial value of the variables were randomly chosen to be false (0) or true (1)
empty_series[:, 1] .= rand(Bool, p)

# Initial value of the variables were randomly chosen to be 0.0 or 1.0 
empty_series[:, n_constant_nodes] .= rand(Bool, p)

# Saving the empty time series as the time series (so it can be regenerated later)
time_series = empty_series

# Saving a network_score matrix to save the scores later
network_scores = zeros(n_obs, 4)

#example mutation_profile to see if function works
mutation_profile = sh["B2"]

"WT1,FLT3,DNMT3A,DNMT3A"

In [29]:
# score_personal function #

#creating the function for personal score
function score_personal(mutation_profile)
    # Separate the values by commas
    mutation_profile = split(mutation_profile, ',')

    # Remove repeated values
    mutation_profile = unique(mutation_profile)
    
    # Join back together
    join(mutation_profile, ",")

    println(mutation_profile)
    println("preparing to create time series")
    
    #doing the constant nodes first (they do not change, but they still need noise)    
    for col in (n_constant_nodes + 1) :n_obs
        #simulate the constant nodes for only a couple of steps
        time_series[1, col] = time_series[1, col - 1]  # KITLG = KITLG
        time_series[2, col] = time_series[2, col - 1]  # FLT3LG = FLT3LG
        time_series[4, col] = time_series[4, col - 1]  # KDM5A = KDM5A
        time_series[7, col] = time_series[7, col - 1]  # CBLB = CBLB
        time_series[12, col] = time_series[12, col - 1]  # CDK1 = CDK1
        time_series[13, col] = time_series[13, col - 1]  # STAG2 = STAG2
        time_series[14, col] = time_series[14, col - 1]  # SH2B3 = SH2B3
        time_series[16, col] = time_series[16, col - 1]  # AML1_ETO = AML1_ETO
        time_series[17, col] = time_series[17, col - 1]  # PML_RARalpha = PML_RARalpha
        time_series[37, col] = time_series[37,col - 1] #NUP98_Fusion = NUP98_Fusion
        time_series[38, col] = time_series[38,col - 1] #MLL_Fusion = MLL_Fusion
        time_series[46, col] = time_series[46,col - 1] #NPM1 = NPM1
        time_series[48, col] = time_series[48,col - 1] #IDH2 = IDH2
        time_series[53, col] = time_series[53,col - 1]	#AMPK = AMPK
        time_series[54, col] = time_series[54,col - 1] #ASXL2 = ASXL2
        time_series[58, col] = time_series[58,col - 1]	#DNMT3A = DNMT3A
        time_series[60, col] = time_series[60,col - 1]	#ASXL1 = ASXL1
        time_series[64, col] = time_series[64,col - 1] #PHF6 = PHF6
        time_series[68, col] = time_series[68,col - 1] #SRSF2 = SRSF2
        time_series[70, col] = time_series[70,col - 1]	#BCOR = BCOR
        time_series[74, col] = time_series[74,col - 1]	#U2AF1 = U2AF1
        time_series[75, col] = time_series[75,col - 1]	#XPO1 = XPO1
        time_series[76, col] = time_series[76,col - 1]	#CREBBP = CREBBP
    
    end
    
      for col in 2:n_obs
        # Simulate each time step based on the previous time step and Boolean functions
        time_series[3,col] = time_series[6,col-1] & time_series[11,col-1] & time_series[10,col-1]	#GRB2 = BCR_ABL | FLT3 | KIT
        time_series[5,col] = !time_series[4,col-1] & time_series[5,col-1] #PTEN = !KDM5A & PTEN
        time_series[6,col] = !time_series[5,col-1] #BCR_ABL = !PTEN
        time_series[8,col] = time_series[3,col-1] #CBL = GRB2
        time_series[9,col] = time_series[3,col-1] #SOS1 = GRB2
        time_series[10,col] = !(time_series[8,col-1] | time_series[7,col-1]) & (time_series[16,col-1] | time_series[1,col-1]) #KIT = !(CBL | CBLB) & (AML1_ETO | KITLG)
        time_series[11,col] = !(time_series[8,col-1] | time_series[7,col-1]) & time_series[2,col-1] #FLT3 = !(CBL | CBLB ) & FLT3LG
        time_series[15,col] = !time_series[11,col-1] #PTPN6 =!FLT3
        time_series[18,col] = time_series[11,col-1] & time_series[10,col-1] #PTPN11 = FLT3 | KIT
        time_series[19,col] = !time_series[12,col-1] #CUX1 =!CDK1
        time_series[20,col] = time_series[13,col-1] #RAD21 = STAG2
        time_series[21,col] = !time_series[14,col-1] & time_series[29,col-1] #BCL2L1 = !SH2B3 & STAT5A
        time_series[22,col] = !(time_series[16,col-1] | time_series[15,col-1] | time_series[14,col-1]) & (time_series[6,col-1] | time_series[10,col-1]) #JAK2 = !(AML1_ETO | PTPN6 | SH2B3) & (BCR_ABL | KIT)
        time_series[23,col] = time_series[17,col-1] #CCNA1 = PML_RARalpha
        time_series[24,col] = !(time_series[16,col-1] | time_series[11,col-1]) | (time_series[17,col-1]) #CEBPA =!(AML1_ETO | FLT3 | PML_RARalpha)
        time_series[25,col] = !time_series[19,col-1] #PIK3IP1 =!CUX1
        time_series[26,col] = time_series[9,col-1] & time_series[18,col-1] #NRAS = SOS1 | PTPN11
        time_series[27,col] = !time_series[20,col-1] & time_series[17,col-1] #GATA2 = !RAD21 & PML_RARalpha
        time_series[28,col] = !time_series[21,col-1] #BAX = !BCL2L1
        time_series[29,col] = time_series[22,col-1] & time_series[6,col-1] #STAT5A = JAK2 | BCR_ABL 
        time_series[30,col] = !time_series[24,col-1] #SOX4 = !CEBPA
        time_series[31,col] = !(time_series[25,col-1]) & (time_series[11,col-1] | time_series[10,col-1] | time_series[26,col-1] | time_series[32,col-1]) #PI3K = !PIK3IP1 & (FLT3 | KIT | NRAS | SPI1)
        time_series[32,col] = !time_series[27,col-1] & time_series[24,col-1] #SPI1 = !GATA2 & CEBPA
        time_series[33,col] = time_series[29,col-1] #PIM = STAT5A
        time_series[34,col] = time_series[30,col-1] & time_series[11,col-1] #CTNNB1 =  SOX4 | FLT3 
        time_series[35,col] = time_series[31,col-1] #AKT = PI3K 
        time_series[36,col] = !time_series[32,col-1] & time_series[27,col-1] #GATA1 = !SPI1 & GATA2
        time_series[39,col] = !time_series[35,col-1] #FOXO = !AKT
        time_series[40,col] = time_series[35,col-1] #EP300 = AKT
        time_series[41,col] = !time_series[35,col-1] & time_series[26,col-1] #BRAF = !AKT & NRAS
        time_series[42,col] = time_series[36,col-1] #CBFB = GATA1
        time_series[43,col] = time_series[37,col-1] #CDK6 = NUP98_Fusion
        time_series[44,col] = time_series[38,col-1] #MECOM = MLL_Fusion
        time_series[45,col] = time_series[38,col-1] #CBFbeta_MYH11 = MLL_Fusion
        time_series[47,col] = time_series[39,col-1] #IDH1 = FOXO
        time_series[49,col] = time_series[41,col-1] #ERK1/2 = BRAF 
        time_series[50,col] = !(time_series[38,col-1] | time_series[45,col-1] | time_series[20,col-1]) & (time_series[11,col-1] | time_series[43,col-1] | time_series[42,col-1] | time_series[50,col-1])	#RUNX1 = !(MLL_Fusion | CBFbeta_MYH11 | RAD21) & (FLT3 | CDK6 | CBFB | RUNX1)
        time_series[51,col] = time_series[45,col-1] #DOT1L = CBFbeta_MYH11
        time_series[52,col] = time_series[46,col-1] #FBXW7 = NPM1
        time_series[55,col] = time_series[47,col-1] & time_series[48,col-1] #OXO2 = IDH1 | IDH2
        time_series[56,col] = !time_series[49,col-1] #ETV6 = !ERK1/2
        time_series[57,col] = time_series[49,col-1] #AP1 = ERK1/2 
        time_series[59,col] = !time_series[35,col-1] #AEZH2 = !AKT
        time_series[61,col] = !(time_series[52,col-1] | time_series[50,col-1]) & (time_series[49,col-1] | time_series[51,col-1]) #MYC = !(FBXW7 | RUNX1) & (ERK1/2 | DOT1L)
        time_series[62,col] = !time_series[53,col-1] & !time_series[35,col-1] #MTOR = !(AMPK | AKT)
        time_series[63,col] = time_series[55,col-1] | time_series[53,col-1] | time_series[54,col-1] #TET2 = OXO2 | AMPK | ASXL2
        time_series[65,col] = !time_series[58,col-1] #CCND1 = !DNMT3A 
        time_series[66,col] = !(time_series[59,col-1] | time_series[58,col-1] | time_series[60,col-1]) & time_series[51,col-1] #HOXA9 = !(AEZH2 | DNMT3A | ASXL1) & DOT1L 
        time_series[67,col] = !(time_series[58,col-1] | time_series[61,col-1] | time_series[16,col-1]) & (time_series[60,col-1] | time_series[46,col-1]) #CDKN2A = !( DNMT3A | MYC | AML1_ETO) & (ASXL1 | NPM1)
        time_series[69,col] = time_series[63,col-1] #WT1 = TET2
        time_series[71,col] = !time_series[64,col-1] #UBTF = !PHF6
        time_series[72,col] = !time_series[58,col-1] & (time_series[66,col-1] | time_series[51,col-1]) #MEIS1 =!DNMT3A & (HOXA9 | DOT1L)
        time_series[73,col] = !time_series[67,col-1] & (time_series[68,col-1] | time_series[77,col-1] | time_series[35,col-1]) #MDM2 = !CDKN2A & (SRSF2 | TP53 | AKT)
        time_series[77,col] = !(time_series[73,col-1] | time_series[45,col-1] | time_series[75,col-1]) & (time_series[74,col-1]) & time_series[76,col-1] #TP53 = !(MDM2 | CBFbeta_MYH11 | XPO1 | U2AF1) & (CREBBP)
        time_series[78,col] = !time_series[77,col-1] & (time_series[49,col-1] | time_series[51,col-1]) #BCL2 = !TP53 & (ERK1/2 | DOT1L)   
        time_series[79,col] = !(time_series[67,col-1] | time_series[69,col-1] | time_series[70,col-1]) & (time_series[29,col-1] | time_series[34,col-1] | time_series[61,col-1] | time_series[57,col-1] | time_series[23,col-1] | time_series[72,col-1] | time_series[65,col-1] | time_series[71,col-1] | time_series[39,col-1]) #Proliferation = !(CDKN2A | WT1 | BCOR) & (STAT5A | CTNNB1 | MYC | AP1 | CCNA1 | MEIS1 | CCND1 | UBTF | FOXO)
        time_series[80,col] = !(time_series[30,col-1] | time_series[45,col-1]) & (time_series[32,col-1] | time_series[50,col-1] | time_series[24,col-1] | time_series[56,col-1] | time_series[40,col-1]) #Differentiation = !(SOX4 | CBFbeta_MYH11) & (SPI1 | RUNX1 | CEBPA | ETV6 | EP300)
        time_series[81,col] = !(time_series[78,col-1] | time_series[33,col-1]) & (time_series[39,col-1] | time_series[77,col-1] | time_series[28,col-1])  #Apoptosis = !(BCL2 | PIM) & (FOXO | TP53 | BAX ) 
        
		#apply random noise
		noise = rand(p) .< chance_to_flip
		time_series[:, col] .= xor.(time_series[:, col], noise)
		
	end 
    
    #for the oncogenes
	for col in 1:n_obs 
		#knocking in genes
		if any(x -> occursin.("KITLG", x), mutation_profile)
		time_series[1,col] = 1
		end
		
		if any(x -> occursin.("FLT3LG", x), mutation_profile)
		time_series[2,col] = 1
		end
		
		if any(x -> occursin.("GRB2", x), mutation_profile)
		time_series[3,col] = 1
		end
		
		if any(x -> occursin.("KDM5A", x), mutation_profile)
		time_series[4,col] = 1
		end
		
		if any(x -> occursin.("BCR_ABL", x), mutation_profile)
		time_series[6,col] = 1
		end
		
		if any(x -> occursin.("SOS1", x), mutation_profile)
		time_series[9,col] = 1
		end
		
		if any(x -> occursin.("KIT", x), mutation_profile)
		time_series[10,col] = 1
		end
		
		if any(x -> occursin.("FLT3", x), mutation_profile)
		time_series[11,col] = 1
		end
		
		if any(x -> occursin.("AML1_ETO", x), mutation_profile)
		time_series[16,col] = 1
		end
		
		if any(x -> occursin.("PML_RARalpha", x), mutation_profile)
		time_series[17,col] = 1	  #check the lettering!
		end
		
		if any(x -> occursin.("PTPN11", x), mutation_profile)
		time_series[18,col] = 1
		end
		
		if any(x -> occursin.("CUX1", x), mutation_profile)
		time_series[19,col] = 1
		end
		
		if any(x -> occursin.("RAD21", x), mutation_profile)
		time_series[20,col] = 1
		end
		
		if any(x -> occursin.("BCL2L1", x), mutation_profile)
		time_series[21,col] = 1
		end
		
		if any(x -> occursin.("JAK2", x), mutation_profile)
		time_series[22,col] = 1
		end
		
		if any(x -> occursin.("CCNA1", x), mutation_profile)
		time_series[23,col] = 1
		end
		
		if any(x -> occursin.("NRAS", x), mutation_profile)
		time_series[26,col] = 1
		end
		
		if any(x -> occursin.("GATA2", x), mutation_profile)
		time_series[27,col] = 1
		end
		
		if any(x -> occursin.("STAT5A", x), mutation_profile)
		time_series[29,col] = 1
		end
		
		if any(x -> occursin.("SOX4", x), mutation_profile)
		time_series[30,col] = 1
		end
		
		if any(x -> occursin.("PI3K", x), mutation_profile)
		time_series[31,col] = 1
		end
		
		if any(x -> occursin.("PIM", x), mutation_profile)
		time_series[33,col] = 1
		end
		
		if any(x -> occursin.("CTNNB1", x), mutation_profile)
		time_series[34,col] = 1
		end
		
		if any(x -> occursin.("AKT", x), mutation_profile)
		time_series[35,col] = 1
		end
		
		if any(x -> occursin.("GATA1", x), mutation_profile)
		time_series[36,col] = 1
		end
		
		if any(x -> occursin.("MLL_Fusion", x), mutation_profile)
		time_series[38,col] = 1 #check the lettering!
		end
		
		if any(x -> occursin.("BRAF", x), mutation_profile)
		time_series[41,col] = 1
		end
		
		if any(x -> occursin.("CDK6", x), mutation_profile)
		time_series[43,col] = 1
		end
		
		if any(x -> occursin.("MECOM", x), mutation_profile)
		time_series[44,col] = 1
		end
		
		if any(x -> occursin.("NPM1", x), mutation_profile)
		time_series[46,col] = 1
		end
		
		if any(x -> occursin.("IDH1", x), mutation_profile)
		time_series[47,col] = 1
		end
		
		if any(x -> occursin.("IDH2", x), mutation_profile)
		time_series[48,col] = 1
		end
		
		if any(x -> occursin.("ERK", x), mutation_profile)
		time_series[49,col] = 1 #check the lettering!
		end
		
		if any(x -> occursin.("DOT1L", x), mutation_profile)
		time_series[51,col] = 1
		end
		
		if any(x -> occursin.("AMPK", x), mutation_profile)
		time_series[53,col] = 1	#check the value
		end
		
		if any(x -> occursin.("OXO2", x), mutation_profile)
		time_series[55,col] = 1 #check the value
		end
		
		if any(x -> occursin.("AP1", x), mutation_profile)
		time_series[57,col] = 1
		end
		
		if any(x -> occursin.("MYC", x), mutation_profile)
		time_series[61,col] = 1
		end
		
		if any(x -> occursin.("MTOR", x), mutation_profile)
		time_series[62,col] = 1 #check the value
		end
		
		if any(x -> occursin.("CCND1", x), mutation_profile)
		time_series[65,col] = 1
		end
		
		if any(x -> occursin.("HOXA9", x), mutation_profile)
		time_series[66,col] = 1
		end
		
		if any(x -> occursin.("SRSF2", x), mutation_profile)
		time_series[68,col] = 1 #check the value
		end
		
		if any(x -> occursin.("UBTF", x), mutation_profile)
		time_series[71,col] = 1
		end
		
		if any(x -> occursin.("MEIS1", x), mutation_profile)
		time_series[72,col] = 1
		end
		
		if any(x -> occursin.("MDM2", x), mutation_profile)
		time_series[73,col] = 1	#check the value
		end
		
		if any(x -> occursin.("U2AF1", x), mutation_profile)
		time_series[74,col] = 1
		end
		
		if any(x -> occursin.("XPO1", x), mutation_profile)
		time_series[75,col] = 1
		end
		
		if any(x -> occursin.("BCL2", x), mutation_profile)
		time_series[78,col] = 1
		end
	end
	
	#for the TSG
	for col in 1:n_obs 
		#knocking in genes
		if any(x -> occursin.("PTEN", x), mutation_profile)
		time_series[5,col] = 0
		end
		
		if any(x -> occursin.("CBLB", x), mutation_profile)
		time_series[7,col] = 0
		end
		
		if any(x -> occursin.("CBL", x), mutation_profile)
		time_series[8,col] = 0
		end
		
		if any(x -> occursin.("CDK1", x), mutation_profile)
		time_series[12,col] = 0
		end
		
		if any(x -> occursin.("STAG2", x), mutation_profile)
		time_series[13,col] = 0
		end
		
		if any(x -> occursin.("SH2B3", x), mutation_profile)
		time_series[14,col] = 0
		end
		
		if any(x -> occursin.("PTPN6", x), mutation_profile)
		time_series[15,col] = 0
		end
		
		if any(x -> occursin.("CEBPA", x), mutation_profile)
		time_series[24,col] = 0
		end
		
		if any(x -> occursin.("PIK3IP1", x), mutation_profile)
		time_series[25,col] = 0
		end
		
		if any(x -> occursin.("BAX", x), mutation_profile)
		time_series[28,col] = 0
		end
		
		if any(x -> occursin.("SPI1", x), mutation_profile)
		time_series[32,col] = 0
		end
		
		if any(x -> occursin.("NUP98_Fusion", x), mutation_profile)
		time_series[37,col] = 0 #check the lettering!
		end
		
		if any(x -> occursin.("FOXO", x), mutation_profile)
		time_series[39,col] = 0
		end
		
		if any(x -> occursin.("EP300", x), mutation_profile)
		time_series[40,col] = 0
		end
		
		if any(x -> occursin.("CBFB", x), mutation_profile)
		time_series[42,col] = 0
		end
		
		if any(x -> occursin.("CBFbeta_MYH11", x), mutation_profile)
		time_series[45,col] = 0 #check the lettering!
		end
		
		if any(x -> occursin.("RUNX1", x), mutation_profile)
		time_series[50,col] = 0
		end
		
		if any(x -> occursin.("FBXW7", x), mutation_profile)
		time_series[52,col] = 0
		end
		
		if any(x -> occursin.("ASXL2", x), mutation_profile)
		time_series[54,col] = 0
		end
		
		if any(x -> occursin.("ETV6", x), mutation_profile)
		time_series[56,col] = 0
		end
		
		if any(x -> occursin.("DNMT3A", x), mutation_profile)
		time_series[58,col] = 0
		end
		
		if any(x -> occursin.("AEZH2", x), mutation_profile)
		time_series[59,col] = 0
		end
		
		if any(x -> occursin.("ASXL1", x), mutation_profile)
		time_series[60,col] = 0
		end
		
		if any(x -> occursin.("TET2", x), mutation_profile)
		time_series[63,col] = 0
		end
		
		if any(x -> occursin.("PHF6", x), mutation_profile)
		time_series[64,col] = 0
		end
		
		if any(x -> occursin.("CDKN2A", x), mutation_profile)
		time_series[67,col] = 0
		end
		
		if any(x -> occursin.("WT1", x), mutation_profile)
		time_series[69,col] = 0
		end
		
		if any(x -> occursin.("BCOR", x), mutation_profile)
		time_series[70,col] = 0
		end
		
		if any(x -> occursin.("CREBBP", x), mutation_profile)
		time_series[76,col] = 0
		end
		
		if any(x -> occursin.("TP53", x), mutation_profile)
		time_series[77,col] = 0
		end
	end
    
	#computing the scores
	for col in 1:n_obs
        network_scores[col,1] = time_series[79,col]
		network_scores[col,2] = time_series[80,col]
		network_scores[col,3] = time_series[81,col]
		network_scores[col,4] = (time_series[79,col] - (time_series[80,col] + time_series[81,col]))
	end  
    
	println("time series is complete")
    
	#computing the average scores from rolling mean
	#rolling window of size n_obs/1000
	window_size = 2000 
	
	#calculate the rolling average for each variable using the RollingFunctions package
	rolling_avg = rollmean(network_scores[:,4], window_size)
	
	#Compute the final_score
    final_score = mean(rolling_avg[(length(rolling_avg)-2000):length(rolling_avg)], dims=1)
	final_score = final_score[1]
	final_score = round(final_score, sigdigits=3)
    
    return final_score
end

score_personal (generic function with 1 method)

In [30]:
mutation_profile = sh["B1:B20"] 

20×1 Matrix{Any}:
 "TET2,TET2,NPM1"
 "WT1,FLT3,DNMT3A,DNMT3A"
 "ETV6,FLT3,SRSF2,SRSF2,SETBP1,SETBP1"
 "FLT3,CBLB,CBLB,TET2,TET2,TET2,TET2,NPM1"
 "WT1,FLT3,DNMT3A,DNMT3A"
 "FLT3,HOXA1"
 "KRAS,KRAS,SOCS1,DNMT3A,NPM1"
 "WT1,WT1,WT1,IDH2,IDH2,KIAA1328,KIAA2026,BCOR"
 "WT1,FLT3,FLT3,IDH2,IDH2,DNMT3A,DNMT3A,NPM1"
 "FLT3,CEBPA,TET2,TET2,NPM1"
 "FLT3,SF3B1,SF3B1,RUNX1,RUNX1"
 "WT1,FLT3,DNMT3A"
 "FLT3,DNMT3A,DNMT3A,IDH1,IDH1,NPM1,BHLHE22"
 "FLT3,FLT3,CNOT3,DNMT3A,DNMT3A,NPM1"
 "WT1,WT1,WT1,IDH2,IDH2,KRTAP4-5,BCOR,PHF6,PHF6,PHF6"
 "WT1,FLT3,FLT3,FLT3,IDH2,IDH2,DNMT3A,DNMT3A,NPM1"
 "WT1,FLT3,FLT3,IDH2,IDH2,DNMT3A,DNMT3A,NPM1"
 "FLT3,SF3B1,SF3B1,RUNX1,RUNX1"
 "FLT3,CBLB,CBLB,TET2,TET2,TET2,NPM1"
 "FLT3,NF1,NF1,DNMT3A,DNMT3A,IDH1,IDH1,FOXP1,FOXP1,NPM1"

In [31]:
mutation_profile[1]

"TET2,TET2,NPM1"

In [32]:
# Using the function #
patient_matrix = zeros(20, 1)
mutation_profile = sh["B1:B20"] #getting all the mutation profiles ONLY

for i in 1:(length(mutation_profile))
	#get the result score
	result = score_personal(mutation_profile[i]) 
	println(result)
	
	patient_matrix[i,1] = result 
	
	time_series = empty_series
end

SubString{String}["TET2", "NPM1"]
preparing to create time series
time series is complete
-0.505
SubString{String}["WT1", "FLT3", "DNMT3A"]
preparing to create time series
time series is complete
-0.607
SubString{String}["ETV6", "FLT3", "SRSF2", "SETBP1"]
preparing to create time series
time series is complete
-0.513
SubString{String}["FLT3", "CBLB", "TET2", "NPM1"]
preparing to create time series
time series is complete
-0.587
SubString{String}["WT1", "FLT3", "DNMT3A"]
preparing to create time series
time series is complete
-0.5
SubString{String}["FLT3", "HOXA1"]
preparing to create time series
time series is complete
-0.556
SubString{String}["KRAS", "SOCS1", "DNMT3A", "NPM1"]
preparing to create time series
time series is complete
-0.543
SubString{String}["WT1", "IDH2", "KIAA1328", "KIAA2026", "BCOR"]
preparing to create time series
time series is complete
-0.517
SubString{String}["WT1", "FLT3", "IDH2", "DNMT3A", "NPM1"]
preparing to create time series
time series is complete
-0.552
